In [224]:
import pandas as pd
import pymorphy2
from gensim.models.word2vec import Word2Vec
from gensim.models import Phrases
from gensim.models import KeyedVectors

df = pd.read_excel("датасет 15072022.xlsx")
df.head()
morph = pymorphy2.MorphAnalyzer()

In [225]:
df_test1 = df[["Группа продукции", "Технические регламенты", "Коды ТН ВЭД ЕАЭС", "Общее наименование продукции"]].copy()
df_test1.head()

,Группа продукции,Технические регламенты,Коды ТН ВЭД ЕАЭС,Общее наименование продукции
0,Узлы пересечения противопожарных преград кабел...,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,4016999708,Проходки кабельные модульные универсальные огн...
1,"Противопожарные окна, двери, двери шахт лифтов...",ТР ЕАЭС 043/2017 О требованиях к средствам обе...,7308909809,Ворота противопожарные металлические распашные...
2,"Клапаны противопожарные нормально открытые, кл...",ТР ЕАЭС 043/2017 О требованиях к средствам обе...,8415900009; 8415900009; 8415900009; 8415900009...,Клапаны противопожарные марки КПВ.01(EI60) пря...
3,Вытяжные вентиляторы,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,8414594000,Вентиляторы крышные радиальные дымоудаления с ...
4,Вытяжные вентиляторы,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,8414594000,Вентиляторы радиальные дымоудаления типов ВР-8...


In [227]:
with open("ТН ВЭД\TNVED\TNVED3.Txt", encoding='cp866') as fout_tnved3:
    dict_group = dict()
    fout_tnved3.readline()
    for line in fout_tnved3.readlines():
        line = line.replace('\xa0', '')
        line = line.replace(',', '')
        ls = line.split('|')
        ls.pop(-1)
        dict_group[ls[0]+ ls[1]] = ls[2]
    lst_names_group = [i.split() for i in dict_group.values()]
    print(lst_names_group)  

[['ЛОШАДИ', 'ОСЛЫ', 'МУЛЫ', 'И', 'ЛОШАКИ', 'ЖИВЫЕ'], ['КРУПНЫЙ', 'РОГАТЫЙ', 'СКОТ', 'ЖИВОЙ'], ['СВИНЬИ', 'ЖИВЫЕ'], ['ОВЦЫ', 'И', 'КОЗЫ', 'ЖИВЫЕ'], ['ДОМАШНЯЯ', 'ПТИЦА', 'ЖИВАЯ', 'ТО', 'ЕСТЬ', 'КУРЫ', 'ДОМАШНИЕ', '(GALLUS', 'DOMESTICUS)', 'УТКИ', 'ГУСИ', 'ИНДЕЙКИ', 'И', 'ЦЕСАРКИ'], ['ЖИВЫЕ', 'ЖИВОТНЫЕ', 'ПРОЧИЕ'], ['МЯСО', 'КРУПНОГО', 'РОГАТОГО', 'СКОТА', 'СВЕЖЕЕ', 'ИЛИ', 'ОХЛАЖДЕННОЕ'], ['МЯСО', 'КРУПНОГО', 'РОГАТОГО', 'СКОТА', 'ЗАМОРОЖЕННОЕ'], ['СВИНИНА', 'СВЕЖАЯ', 'ОХЛАЖДЕННАЯ', 'ИЛИ', 'ЗАМОРОЖЕННАЯ'], ['БАРАНИНА', 'ИЛИ', 'КОЗЛЯТИНА', 'СВЕЖАЯ', 'ОХЛАЖДЕННАЯ', 'ИЛИ', 'ЗАМОРОЖЕННАЯ'], ['ПИЩЕВЫЕ', 'СУБПРОДУКТЫ', 'КРУПНОГО', 'РОГАТОГО', 'СКОТА', 'СВИНЕЙ', 'ОВЕЦ', 'КОЗ', 'ЛОШАДЕЙ', 'ОСЛОВ', 'МУЛОВ', 'ИЛИ', 'ЛОШАКОВ', 'СВЕЖИЕ', 'ОХЛАЖДЕННЫЕ', 'ИЛИ', 'ЗАМОРОЖЕННЫЕ'], ['МЯСО', 'И', 'ПИЩЕВЫЕ', 'СУБПРОДУКТЫ', 'ДОМАШНЕЙ', 'ПТИЦЫ', 'УКАЗАННОЙ', 'В', 'ТОВАРНОЙ', 'ПОЗИЦИИ', '0105', 'СВЕЖИЕ', 'ОХЛАЖДЕННЫЕ', 'ИЛИ', 'ЗАМОРОЖЕННЫЕ'], ['ПРОЧИЕ', 'МЯСО', 'И', 'ПИЩЕВЫЕ', 'МЯСНЫЕ', 'СУБПРОДУКТЫ', 'СВЕЖИЕ'

In [228]:
with open("ТН ВЭД\TNVED\TNVED2.Txt", encoding='cp866') as fout_tnved2:
    dict_common = dict()
    fout_tnved2.readline()
    for line in fout_tnved2.readlines():
        line = line.replace('\xa0', '')
        line = line.replace(',', '')
        ls = line.split('|')
        ls.pop(-1)
        dict_common[ls[0] + ls[1]] = ls[2]
    lst_names_common = [i.split() for i in dict_common.values()]
    print(lst_names_common)

[['ЖИВЫЕ', 'ЖИВОТНЫЕ'], ['МЯСО', 'И', 'ПИЩЕВЫЕ', 'МЯСНЫЕ', 'СУБПРОДУКТЫ'], ['РЫБА', 'И', 'РАКООБРАЗНЫЕ', 'МОЛЛЮСКИ', 'И', 'ПРОЧИЕ', 'ВОДНЫЕ', 'БЕСПОЗВОНОЧНЫЕ'], ['МОЛОЧНАЯ', 'ПРОДУКЦИЯ;', 'ЯЙЦА', 'ПТИЦ;', 'МЕД', 'НАТУРАЛЬНЫЙ;', 'ПИЩЕВЫЕ', 'ПРОДУКТЫ', 'ЖИВОТНОГО', 'ПРОИСХОЖДЕНИЯ', 'В', 'ДРУГОМ', 'МЕСТЕ', 'НЕ', 'ПОИМЕНОВАННЫЕ', 'ИЛИ', 'НЕ', 'ВКЛЮЧЕННЫЕ'], ['ПРОДУКТЫ', 'ЖИВОТНОГО', 'ПРОИСХОЖДЕНИЯ', 'В', 'ДРУГОМ', 'МЕСТЕ', 'НЕ', 'ПОИМЕНОВАННЫЕ', 'ИЛИ', 'НЕ', 'ВКЛЮЧЕННЫЕ'], ['ЖИВЫЕ', 'ДЕРЕВЬЯ', 'И', 'ДРУГИЕ', 'РАСТЕНИЯ;', 'ЛУКОВИЦЫ', 'КОРНИ', 'И', 'ПРОЧИЕ', 'АНАЛОГИЧНЫЕ', 'ЧАСТИ', 'РАСТЕНИЙ;', 'СРЕЗАННЫЕ', 'ЦВЕТЫ', 'И', 'ДЕКОРАТИВНАЯ', 'ЗЕЛЕНЬ'], ['ОВОЩИ', 'И', 'НЕКОТОРЫЕ', 'СЪЕДОБНЫЕ', 'КОРНЕПЛОДЫ', 'И', 'КЛУБНЕПЛОДЫ'], ['СЪЕДОБНЫЕ', 'ФРУКТЫ', 'И', 'ОРЕХИ;', 'КОЖУРА', 'ЦИТРУСОВЫХ', 'ПЛОДОВ', 'ИЛИ', 'КОРКИ', 'ДЫНЬ'], ['КОФЕ', 'ЧАЙ', 'МАТЕ', 'ИЛИ', 'ПАРАГВАЙСКИЙ', 'ЧАЙ', 'И', 'ПРЯНОСТИ'], ['ЗЛАКИ'], ['ПРОДУКЦИЯ', 'МУКОМОЛЬНО-КРУПЯНОЙ', 'ПРОМЫШЛЕННОСТИ;', 'СОЛОД;', 'КРАХМАЛЫ;', 'ИНУЛИН;', 'ПШ

In [229]:
lexemes = [morph.parse(j)[0].lexeme for i in lst_names_group for j in i]
voc_lex_group = [j.word.upper() for i in lexemes for j in i]
print(voc_lex_group[0], voc_lex_group[1])

ЛОШАДЬ ЛОШАДИ


In [260]:
model = Word2Vec(lst_names_group, min_count=1)
model.build_vocab(voc_lex_group)
for i in range(200):
    model.train(lst_names_common, total_examples=model.corpus_count, compute_loss=True, epochs=model.epochs)
print(model.get_latest_training_loss())
word_vectors.save("word2vec.wordvectors")
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')
for i in wv.most_similar(positive=['ЖИВЫЕ'], topn=10):
    print(i)

233.101318359375
('ИНУЛИН;', 0.26407158374786377)
('СОЕДИНЕНИЯ', 0.2588173449039459)
('ЯДЕРНЫЕ,', 0.2356453686952591)
('МОНЕТЫ', 0.2345217764377594)
('РАСТЕНИЯ;', 0.21936844289302826)
('ФОТО-', 0.20481610298156738)
('КОНДИТЕРСКИЕ', 0.19953595101833344)
('НЕГО', 0.19941313564777374)
('СВЕТОВЫЕ', 0.196404367685318)
('НЕОЧИЩЕННЫЙ;', 0.19615767896175385)
